In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import re
from datetime import date
from datetime import datetime

######notas:
######Hay que renombrar las variables para conectar las funciones de import, acquire y las handling:

###: Exportadores

#Import function:

def _import_():
    df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')
    df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')
    df = sqlite3.connect('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

    return

def _acquire_():
    dfBI = pd.read_sql_query("SELECT * FROM business_info", df)
    dfRI = pd.read_sql_query("SELECT * FROM rank_info", df)
    dfPI = pd.read_sql_query("SELECT * FROM personal_info", df)


#EXTRACT PRIMITIVE DATA:

#BUSINES INFO

    
#WRANGLING bussines_info
    #drop realTimeWorth - 100% nulls

dfBI.drop(['realTimeWorth'], axis='columns', inplace=True)

    #eliminating columns that do not add value (Unnamed:0, realTimePosition)
dfBI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfBI.drop(['realTimePosition'], axis='columns', inplace=True)

dfBI.replace({"BUSD": ' '})

    #replacement of 'na' values by 0.0 in wortChange column

dfBI.worthChange.replace('na', '0.0', regex=True, inplace=True)

    #regex - separate numerical values
dfBI.worth.replace('[BUSD]+$', '', regex=True, inplace=True)
dfBI.worthChange.replace('[millions USD]+$', '', regex=True, inplace=True)

    #replacement float values with int
dfBI['worth'] = dfBI['worth'].astype('float').astype('int')
dfBI['worthChange'] = dfBI['worthChange'].astype('float').astype('int')
dfBI['id'] = dfBI['id'].astype('object').astype('int')

    #checking the type of values in columns
print(dfBI['worthChange'].dtype)
print(dfBI['worth'].dtype)
print(dfBI['id'].dtype)

    #Analyzing worthChange column composition:
print(dfBI['worthChange'].value_counts())
print(dfBI['worthChange'].describe())

#Once the value_counts () and describe () methods have been performed, we can see that the column is composed of 97.59% of 0 values, however we can see that they include very extreme values in minimums and maximums, and since it is a column That refers to the exchange rates I think it is convenient not to eliminate the column for futures and possible analyzes during the project.

#I rename columns to indicate the units in which the values are expressed and not to contaminate the dataset.

dfBI = dfBI.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})

    #Make sure there are no duplicate items
dfBI = dfBI.drop_duplicates()

#RESET INDEX

dfBI = dfBI.reset_index(drop=True)

#Exporting the clean dataset to the data/processed folder

###dfBI.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/dfBussinesInfoCLEAN.csv', index=False)


#RANK INFO




#Wrangling rank_info
    #eliminating columns that do not add value (Unnamed:0)

dfRI.drop(['Unnamed: 0'], axis='columns', inplace=True)

    #Wrangling - "position column"

print(dfRI.position.isnull().sum())   #Only 52 NaN values, not proceed delete column

dfRI['position'] = dfRI['position'].fillna(0)   #replace NaN values with 0.0
dfRI['position'] = dfRI['position'].astype('float').astype('int')  #replacement float values with int


#convert all characters to lowercase in column name

dfRI = dfRI.apply(lambda x: x.astype(str).str.lower())


#analyzing types of values to see the next cleaning stage
     #we can verify that we have duplicate elements so we must eliminate them but taking care that we eliminate the correct one and that it contributes less value to the dataframe. 

    #changing object types
    
dfRI['position'] = dfRI['position'].astype('object').astype('int')
dfRI['id'] = dfRI['id'].astype('object').astype('int')

print(dfRI['position'].dtype)
print(dfRI['id'].dtype)

#I delete the rows that have a value of 0 in the position column, they do not add value to the analysis and thus the size of the table is matched with that of Bussines_info
dfRI = dfRI.drop(dfRI[dfRI['position']==0].index)

#RESET INDEX
dfRI = dfRI.reset_index(drop=True)

#Exporting the clean dataset to the data/processed folder

###dfRI.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/dfrankinfoCLEAN.csv', index=False)


#PERSONAL INFO:





#Wrangling personal_info
    #Drop Unnamed:0 and lastName, I delete lastname column since we already have the full name and surname in rank_info
    
dfPI.drop(['Unnamed: 0'], axis='columns', inplace=True)
dfPI.drop(['lastName'], axis='columns', inplace=True)


    #Wrangling gender column
    
dfPI.gender.replace('[M]+$', 'Male', regex=True, inplace=True)
dfPI.gender.replace('[F]+$', 'Female', regex=True, inplace=True)
dfPI['gender'] = dfPI['gender'].astype('object').astype('str')
#print(dfPI['gender'].value_counts())
   
    #Wrangling age column
        #Correct info in age column: delete "years old" and change data types
        
dfPI.age.replace('[years old]+$', '', regex=True, inplace=True)   
dfPI['age'] = dfPI['age'].fillna(0)   #replace NaN values with 0.0
dfPI['age'] = dfPI['age'].astype('float').astype('int') 
#print(dfPI['age'].value_counts())    
#print(dfPI['age'].dtype)  

#Checking Types: 

#print(dfPI['id'].dtype) 
#print(dfPI['age'].dtype)

    #Datetime - preparing 

now = datetime.now()
now.year

    #Checking Outliers: 

stats = dfPI.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

outliers = pd.DataFrame(columns=dfPI.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = dfPI[(dfPI[col] < lower) | 
                   (dfPI[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    'sort=False'
     
dfPI['new_age'] = now.year - outliers['age']

dfPI.new_age.fillna(dfPI.age, inplace=True)

dfPI['age'] = dfPI['new_age']
dfPI.drop(['new_age'], axis='columns', inplace=True)
dfPI.age.replace(now.year, 'None', regex=True, inplace=True)


#Checking Duplicates

dfPI = dfPI.drop_duplicates()

#RESET INDEX

dfPI = dfPI.reset_index(drop=True)

#Exporting the clean dataset to the data/processed folder

###dfPI.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/processed/dfPersonalInfoCLEAN.csv', index=False)
  
    



